In [1]:
import torch 
from torch import nn
from PIL import Image
from torch.utils.data import DataLoader, random_split, Subset
import torchvision.transforms as transforms
from torchvision.transforms import AutoAugment, AutoAugmentPolicy
from torchvision import datasets
import os
from tqdm.auto import tqdm
from timeit import default_timer as timer
from torchvision.models import resnet50, ResNet50_Weights
import sys
sys.path.append('/home/kadenw/Workspace/dog_scanner/dog_scanner')
from dog_scanner import (
    plot_loss_curves,
    train_step,
    test_step,
    train
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

sdogs_path = '/home/kadenw/Workspace/dog_scanner/data/stanford_dogs/images/Images'

IMAGE_SIZE = (224, 224)

aug_transform = transforms.Compose([
    transforms.Resize(size = IMAGE_SIZE),
    transforms.TrivialAugmentWide(), 
   #transforms.AutoAugment(policy=AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor()
])
orig_transform = transforms.Compose([
    transforms.Resize(size = IMAGE_SIZE),
    transforms.ToTensor()
])

data = datasets.ImageFolder(root = sdogs_path,
                            transform = orig_transform,
                            target_transform = None)
total_size = len(data)
train_size = int(total_size*0.8)
test_size = total_size-train_size
train_data, test_data = random_split(data, [train_size, test_size])

aug_train_data = Subset(datasets.ImageFolder(root=sdogs_path, 
                                       transform=aug_transform),
                                       train_data.indices)
comb = torch.utils.data.ConcatDataset([train_data, aug_train_data])

NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 32

train_dataloader = DataLoader(dataset = comb, 
                              batch_size=BATCH_SIZE, 
                              num_workers=NUM_WORKERS, 
                              shuffle=True)
test_dataloader = DataLoader(dataset = test_data, 
                             batch_size=BATCH_SIZE, 
                             num_workers=NUM_WORKERS, 
                             shuffle=False)

cuda
